# Data Wrangling with Spark
This workspace contains a tiny subset (128MB) of the full dataset available (12GB) to explore a smaller subset with Spark before deploying your cluster on the cloud. The dataset describes log events comming from a music streaming service. The records describe events such as logging in to the site, visiting a page, listening to the next song, seeing an ad.

In [45]:
!pip install --upgrade matplotlib

Requirement already up-to-date: matplotlib in /opt/conda/lib/python3.6/site-packages (3.3.4)


In [46]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import max as Fmax
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.types import IntegerType
from pyspark.sql import Window
from pyspark.sql.types import*

import datetime

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
# create a Spark session
spark = SparkSession.builder.appName("Spark SQL").getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [48]:
# the IP address of the master node
path = "mini_sparkify_event_data.json"

In [49]:
# loading the json file into a spark DataFrame
user_log = spark.read.json(path)

# Data Exploration 

In [50]:
# Seeing the columns we have in this dataframe
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



There are fields describing the user like firstName, lastName and userId. We also have information about the request, for example, the page the user accessed, the HTTP method, and the status of the request.

In [51]:
# looking at one particular record
user_log.show(n=1)

+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
|        artist|     auth|firstName|gender|itemInSession|lastName|   length|level|       location|method|    page| registration|sessionId|     song|status|           ts|           userAgent|userId|
+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
|Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|Bakersfield, CA|   PUT|NextSong|1538173362000|       29|Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
only showi

Here we can see that Colin was listening to "Rockpools"

In [52]:
# Looking at the first 5 records
user_log.take(5)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9'),
 Row(artist='Adam Lambert', auth='Logged In', firstName='Colin', gender='M', itemInSession=51, lastName='Freeman', length=282.8273, level='paid', location='

In [53]:
# Summary statistics on the length column
user_log.describe("length").show()

+-------+-----------------+
|summary|           length|
+-------+-----------------+
|  count|           228108|
|   mean|249.1171819778458|
| stddev|99.23517921058361|
|    min|          0.78322|
|    max|       3024.66567|
+-------+-----------------+



In [54]:
print("We have {} rows on the dataset".format(user_log.count()))

We have 286500 rows on the dataset


In [55]:
# Looking at the page requests that we have
user_log.select("page").dropDuplicates().sort("page").show(25)

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|               Login|
|              Logout|
|            NextSong|
|            Register|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
| Submit Registration|
|      Submit Upgrade|
|         Thumbs Down|
|           Thumbs Up|
|             Upgrade|
+--------------------+



Users can login and out, visit the homepage, or play a song.

In [56]:
user_log.select("level").dropDuplicates().sort("level").show()

+-----+
|level|
+-----+
| free|
| paid|
+-----+



We have two subscriptions levels, and we are going to define that a user chured when the changed from paid to free user

In [57]:
user_log.select("auth").dropDuplicates().sort("auth").show() 

+----------+
|      auth|
+----------+
| Cancelled|
|     Guest|
| Logged In|
|Logged Out|
+----------+



In [58]:
# Looking at the events of userId 30
user_log.select(["userId", "firstName", "page", "song"]).filter(user_log.userId == "30").head(10)

[Row(userId='30', firstName='Colin', page='NextSong', song='Rockpools'),
 Row(userId='30', firstName='Colin', page='NextSong', song='Time For Miracles'),
 Row(userId='30', firstName='Colin', page='NextSong', song='Harder Better Faster Stronger'),
 Row(userId='30', firstName='Colin', page='NextSong', song='Passengers (Old Album Version)'),
 Row(userId='30', firstName='Colin', page='Add to Playlist', song=None),
 Row(userId='30', firstName='Colin', page='NextSong', song='Fuck Kitty'),
 Row(userId='30', firstName='Colin', page='NextSong', song='Jade'),
 Row(userId='30', firstName='Colin', page='NextSong', song='So-Called Friends'),
 Row(userId='30', firstName='Colin', page='NextSong', song='Represent'),
 Row(userId='30', firstName='Colin', page='NextSong', song='Here I Am')]

As users use different pages of the application, their page request are logged

# Drop Rows with Missing Values
Dropping missing values, making sure that we have the userId and SessionId for all the records

In [59]:
# Dropping missing values
user_log_valid = user_log.dropna(how="any", subset=["userId", "sessionId"])

In [60]:
# Making sure that we have the userId and SessionId for all the records
user_log.select("userId").dropDuplicates().sort("userId").show()

+------+
|userId|
+------+
|      |
|    10|
|   100|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
|100016|
|100017|
+------+
only showing top 20 rows



There are not None values but there are users with no userId

In [61]:
# Looking at logs of users with no userId
user_log.select(["userId", "firstName", "page", "song"]).filter(user_log.userId == "").collect()

[Row(userId='', firstName=None, page='Home', song=None),
 Row(userId='', firstName=None, page='Help', song=None),
 Row(userId='', firstName=None, page='Home', song=None),
 Row(userId='', firstName=None, page='Login', song=None),
 Row(userId='', firstName=None, page='Home', song=None),
 Row(userId='', firstName=None, page='Login', song=None),
 Row(userId='', firstName=None, page='Login', song=None),
 Row(userId='', firstName=None, page='Login', song=None),
 Row(userId='', firstName=None, page='Home', song=None),
 Row(userId='', firstName=None, page='Login', song=None),
 Row(userId='', firstName=None, page='Home', song=None),
 Row(userId='', firstName=None, page='Home', song=None),
 Row(userId='', firstName=None, page='Login', song=None),
 Row(userId='', firstName=None, page='Home', song=None),
 Row(userId='', firstName=None, page='About', song=None),
 Row(userId='', firstName=None, page='Home', song=None),
 Row(userId='', firstName=None, page='Home', song=None),
 Row(userId='', firstNam

In [62]:
# filter for users with blank user id
blank_pages = user_log.filter(user_log.userId == '') \
    .select(col('page') \
    .alias('blank_pages')) \
    .dropDuplicates()

blank_pages.collect()

[Row(blank_pages='Home'),
 Row(blank_pages='About'),
 Row(blank_pages='Submit Registration'),
 Row(blank_pages='Login'),
 Row(blank_pages='Register'),
 Row(blank_pages='Help'),
 Row(blank_pages='Error')]

It seems that the user's with no id represent users who have not signed up yet or who are signed out and are about to log in.
It may also represents users who got an error while registrating or loging in.

In [63]:
user_log_valid = user_log_valid.filter(user_log_valid["userId"] != "")

In [64]:
user_log_valid.count()

278154

We lost 8346 records. 

In [65]:
print("We lost {} records".format(user_log.count() - user_log_valid.count()))

We lost 8346 records


In [66]:
female_users = user_log_valid.filter(user_log_valid.gender == 'F') \
    .select('userId', 'gender') \
    .dropDuplicates() \
    .count()

male_users = user_log_valid.filter(user_log_valid.gender == 'M') \
    .select('userId', 'gender') \
    .dropDuplicates() \
    .count()

In [67]:
print("We have {} female users and {} male users".format(female_users, male_users))

We have 104 female users and 121 male users


How many songs were played from the most played artist?

In [68]:
user_log_valid.filter(user_log_valid.page == 'NextSong') \
    .select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(desc('Artistcount')) \
    .show(1)

+-------------+-----------+
|       Artist|Artistcount|
+-------------+-----------+
|Kings Of Leon|       1841|
+-------------+-----------+
only showing top 1 row



# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Users Downgrade Their Accounts
We want to distinguish user's activity before and after they downgraded their account from a paid one to a free one.

In [69]:
# Users who downgraded their service
user_log_valid.filter("page = 'Submit Downgrade'").show()

+------+---------+---------+------+-------------+---------+------+-----+--------------------+------+----------------+-------------+---------+----+------+-------------+--------------------+------+
|artist|     auth|firstName|gender|itemInSession| lastName|length|level|            location|method|            page| registration|sessionId|song|status|           ts|           userAgent|userId|
+------+---------+---------+------+-------------+---------+------+-----+--------------------+------+----------------+-------------+---------+----+------+-------------+--------------------+------+
|  null|Logged In|     Kael|     M|           47|    Baker|  null| paid|Kingsport-Bristol...|   PUT|Submit Downgrade|1533102330000|      249|null|   307|1538393619000|"Mozilla/5.0 (Mac...|   131|
|  null|Logged In|   Calvin|     M|           17| Marshall|  null| paid|      Pittsburgh, PA|   PUT|Submit Downgrade|1537120757000|      313|null|   307|1538516445000|"Mozilla/5.0 (Mac...|    38|
|  null|Logged In|  

In [70]:
# Listing the activity of user 61 who downgraded their service
user_log_valid.select(["userId", "firstname", "page", "level", "song"]).filter(user_log_valid.userId == "61").collect()

[Row(userId='61', firstname='Parker', page='Home', level='free', song=None),
 Row(userId='61', firstname='Parker', page='NextSong', level='free', song='Point And Shoot (Explicit Album Version)'),
 Row(userId='61', firstname='Parker', page='NextSong', level='free', song='When A Man Loves A Woman'),
 Row(userId='61', firstname='Parker', page='NextSong', level='free', song='Ungod'),
 Row(userId='61', firstname='Parker', page='NextSong', level='free', song='Matricide'),
 Row(userId='61', firstname='Parker', page='NextSong', level='free', song='Simple'),
 Row(userId='61', firstname='Parker', page='Roll Advert', level='free', song=None),
 Row(userId='61', firstname='Parker', page='NextSong', level='free', song='Close (To The Edit)'),
 Row(userId='61', firstname='Parker', page='Thumbs Up', level='free', song=None),
 Row(userId='61', firstname='Parker', page='NextSong', level='free', song='The Warm Up'),
 Row(userId='61', firstname='Parker', page='NextSong', level='free', song='Vivir_ Amar_ So

After user 61 submited the downgrade she became a free user.

We want to create a phase column which divides the events of a particular user based on these special events. User inventory should have a different value in the phase column before and after she downgraded her service.

In [71]:
# Flaging log entries when users downgrade their accounts
# The value here will be 1 if the event refers to a downgrade, and zero other wise
flag_downgrade_event = udf(lambda x: 1 if x == "Submit Downgrade" else 0, IntegerType())
user_log_valid = user_log_valid.withColumn("downgraded", flag_downgrade_event("page"))

In [72]:
user_log_valid.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', downgraded=0)

The column downgraded appeared last

In [73]:
# Sorting records for a particular user in reverse time order, and adding up the values to distinguish each user's data 
# as either pre or post downgrade events
windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)
# Creating a phase column that uses the window function and cumulative sum
user_log_valid = user_log_valid.withColumn("phase", Fsum("downgraded").over(windowval))

In [74]:
user_log_valid.select(["userId", "firstname", "ts", "page", "level", "phase"]).filter(user_log_valid.userId == "30").sort("ts").collect()

[Row(userId='30', firstname='Colin', ts=1538352117000, page='NextSong', level='paid', phase=1),
 Row(userId='30', firstname='Colin', ts=1538352394000, page='NextSong', level='paid', phase=1),
 Row(userId='30', firstname='Colin', ts=1538352676000, page='NextSong', level='paid', phase=1),
 Row(userId='30', firstname='Colin', ts=1538352899000, page='NextSong', level='paid', phase=1),
 Row(userId='30', firstname='Colin', ts=1538352905000, page='Add to Playlist', level='paid', phase=1),
 Row(userId='30', firstname='Colin', ts=1538353084000, page='NextSong', level='paid', phase=1),
 Row(userId='30', firstname='Colin', ts=1538353218000, page='NextSong', level='paid', phase=1),
 Row(userId='30', firstname='Colin', ts=1538353441000, page='NextSong', level='paid', phase=1),
 Row(userId='30', firstname='Colin', ts=1538353687000, page='NextSong', level='paid', phase=1),
 Row(userId='30', firstname='Colin', ts=1538353909000, page='NextSong', level='paid', phase=1),
 Row(userId='30', firstname='Coli

Looking at the records, all the events before the downgrade event belong to phase one and after they submit downgrade, we switched to phase zero. If we have more than one phase we will see the phase values decreasing from n to zero.

In [75]:
user_log_valid.select("phase").dropDuplicates().sort("phase").show()

+-----+
|phase|
+-----+
|    0|
|    1|
|    2|
|    3|
+-----+



We want to create a Churn column which divides the users who have confirm a cancellation to use as the label of the model.

In [76]:
# Adding a new column called Churn to our dataframe
# The value here will be 1 if the user cancelled, and zero other wise

flag_downgrade_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())
user_log_valid = user_log_valid.withColumn("Churn", flag_downgrade_event("page"))

In [77]:
# # create churn user column
user_log_valid = user_log_valid.withColumn('churn_user', Fmax(
    'Churn').over(Window.partitionBy('userId')))

In [78]:
user_log_valid.groupBy('churn_user').count().show()

+----------+------+
|churn_user| count|
+----------+------+
|         1| 44864|
|         0|233290|
+----------+------+



In [79]:
user_log_valid.head()

Row(artist=None, auth='Logged In', firstName='Darianna', gender='F', itemInSession=34, lastName='Carpenter', length=None, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='Logout', registration=1538016340000, sessionId=187, song=None, status=307, ts=1542823952000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', downgraded=0, phase=0, Churn=0, churn_user=0)

In [80]:
# Aggregates, how much of a specific action they experienced per a certain time unit
# +--------------------+
# |                page|
# +--------------------+
# |          Add Friend|
# |     Add to Playlist|
# |               Error|
# |                Help|
# |               Login|
# |              Logout|
# |            NextSong|
# |            Register|
# |         Roll Advert|
# |       Save Settings|
# |            Settings|
# |         Thumbs Down|
# |           Thumbs Up|
# +--------------------+

#  |-- artist: string (nullable = true)
#  |-- auth: string (nullable = true)
#  |-- firstName: string (nullable = true)
#  |-- gender: string (nullable = true)
#  |-- itemInSession: long (nullable = true)
#  |-- lastName: string (nullable = true)
#  |-- length: double (nullable = true)
#  |-- level: string (nullable = true)
#  |-- location: string (nullable = true)
#  |-- method: string (nullable = true)
#  |-- page: string (nullable = true)
#  |-- registration: long (nullable = true)
#  |-- sessionId: long (nullable = true)
#  |-- song: string (nullable = true)
#  |-- status: long (nullable = true)
#  |-- ts: long (nullable = true)
#  |-- userAgent: string (nullable = true)
#  |-- userId: string (nullable = true)

# Calculating Statistics by Hour
How many songs do users listen to in a given hour?

In [81]:
# Converting timestamps to datetime from epoch time, to get the hour of the day
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).hour)
# Adding a new column called hour to our dataframe
user_log_valid = user_log_valid.withColumn("hour", get_hour(user_log_valid.ts))

get_day = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).day)
user_log_valid = user_log_valid.withColumn("day", get_day(user_log_valid.ts))

# get_weekday = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).weekday)
# user_log_valid = user_log_valid.withColumn("weekday", get_weekday(user_log_valid.ts))

get_month = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).month)
user_log_valid = user_log_valid.withColumn("month", get_month(user_log_valid.ts))

In [82]:
user_log_valid.head()

Row(artist=None, auth='Logged In', firstName='Darianna', gender='F', itemInSession=34, lastName='Carpenter', length=None, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='Logout', registration=1538016340000, sessionId=187, song=None, status=307, ts=1542823952000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', downgraded=0, phase=0, Churn=0, churn_user=0, hour='18', day='21', month='11')

Colin was listening to Martha Tilston at 12am

In [ ]:
# Counting the next song page requests
songs_in_hour = user_log_valid.filter((user_log_valid.page == "NextSong") & (user_log_valid.churn_user == 0)).groupby(user_log_valid.hour).count().orderBy(user_log_valid.hour.cast("float"))
songs_in_hour_pd = songs_in_hour.toPandas()
plt.scatter(songs_in_hour_pd["hour"], songs_in_hour_pd["count"])
plt.xlim(-1, 24)
# plt.ylim(0, 1.2 * max(songs_in_hour_pd["count"]))
plt.xlabel("Hour")
plt.ylabel("Songs Played");

In [ ]:
# Counting the next song page requests
songs_in_hour = user_log_valid.filter((user_log_valid.page == "NextSong") & (user_log_valid.churn_user == 1)).groupby(user_log_valid.hour).count().orderBy(user_log_valid.hour.cast("float"))
songs_in_hour_pd = songs_in_hour.toPandas()
plt.scatter(songs_in_hour_pd["hour"], songs_in_hour_pd["count"])
plt.xlim(-1, 24)
# plt.ylim(0, 1.2 * max(songs_in_hour_pd["count"]))
plt.xlabel("Hour")
plt.ylabel("Songs Played");

We can see some trends in how many songs our users played in different hours of the day

In [ ]:
# # Counting the help page requests
# help_in_hour = user_log_valid.filter((user_log_valid.page == "Help") & (user_log_valid.churn_user == 0)).groupby(user_log_valid.hour).count().orderBy(user_log_valid.hour.cast("float"))
# help_in_hour_pd = help_in_hour.toPandas()
# plt.scatter(help_in_hour_pd["hour"], help_in_hour_pd["count"])
# plt.xlim(-1, 24)
# # plt.ylim(0, 1.2 * max(songs_in_hour_pd["count"]))
# plt.xlabel("Hour")
# plt.ylabel("Helps Requested");

In [ ]:
# # Counting the help page requests
# help_in_hour_chrun = user_log_valid.filter((user_log_valid.page == "Help") & (user_log_valid.churn_user == 1)).groupby(user_log_valid.hour).count().orderBy(user_log_valid.hour.cast("float"))
# help_in_hour_churn_pd = help_in_hour_chrun.toPandas()
# plt.scatter(help_in_hour_churn_pd["hour"], help_in_hour_churn_pd["count"])
# plt.xlim(-1, 24)
# # plt.ylim(0, 1.2 * max(songs_in_hour_pd["count"]))
# plt.xlabel("Hour")
# plt.ylabel("Helps Requested");

In [ ]:
# # Counting the regsitration page requests
# register_in_hour = user_log_valid.filter((user_log_valid.page == "Error") & (user_log_valid.churn_user == 0)).groupby(user_log_valid.hour).count().orderBy(user_log_valid.hour.cast("float"))
# register_in_hour_pd = register_in_hour.toPandas()
# register_in_hour_pd

In [ ]:
# login log out per day

In [ ]:
# Ratio of thumbs down over thumbs up
user_log_valid.filter((user_log_valid.churn_user == 0) & ((user_log_valid.page == 'Thumbs Up') | (user_log_valid.page == 'Thumbs Down'))).groupby("page").count().show()

In [ ]:
# Ratio of thumbs down over thumbs up
user_log_valid.filter((user_log_valid.churn_user == 1) & ((user_log_valid.page == 'Thumbs Up') | (user_log_valid.page == 'Thumbs Down'))).groupby("page").count().show()

In [ ]:
# avg count of each page visit from the total pages visited
user_log_valid.filter(user_log_valid.churn_user == 0).groupby("page").agg({'page':'count'}).show()

In [ ]:
# avg count of each page visit from the total pages visited
user_log_valid.filter(user_log_valid.churn_user == 1).groupby("page").agg({'page':'count'}).show()

In [ ]:
# Aggregates, number of songs

In [ ]:
# How many songs do notChurn users listen to on average between visiting our home page? 
function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = user_log_valid.filter((user_log_valid.page == 'NextSong') | (user_log_valid.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .filter(user_log_valid.churn_user == 0) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

In [ ]:
# How many songs do Churn users listen to on average between visiting our home page? 
cusum.filter((cusum.page == 'NextSong')) \
    .filter(user_log_valid.churn_user == 1) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

In [ ]:
user_log_valid.head(1)

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.